# Montant des transactions immobilières & présence de gares TGV

## Plan du projet 

I. Nettoyage base de données
II. Extraire listes des gares SNCF françaises avec leur code postal 
III. Créer une colonne dans le dataset : gare dans la même commune

In [3]:
import pandas as pd

FILE_PATH = "/home/onyxia/Projet-Python-2A-ENSAE/ValeursFoncieres-2025-S1.csv"

# --- LECTURE DU FICHIER ---
df = pd.read_csv(
    FILE_PATH,
    sep="|",
    dtype=str,                 # On lit tout en string pour éviter les erreurs
    decimal=",",               # Gère les valeurs foncières 468000,00
    na_values=["", " "],       # Traite les champs vides comme NaN
    engine="python"
)

# --- NETTOYAGE MINIMUM ---
# Conversion des dates
df["Date mutation"] = pd.to_datetime(df["Date mutation"], format="%d/%m/%Y", errors="coerce")

# Valeur foncière → float
df["Valeur fonciere"] = (
    df["Valeur fonciere"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

# Surface terrain
df["Surface terrain"] = pd.to_numeric(df["Surface terrain"], errors="coerce")

# Surface réelle bâtie
df["Surface reelle bati"] = pd.to_numeric(df["Surface reelle bati"], errors="coerce")

# Nombre pièces
df["Nombre pieces principales"] = pd.to_numeric(df["Nombre pieces principales"], errors="coerce")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_rows", 20)


print(df.head())
print(df.columns)
print(f"Nombre de lignes : {len(df)}")


  Identifiant de document Reference document 1 Articles CGI 2 Articles CGI 3 Articles CGI 4 Articles CGI 5 Articles CGI No disposition Date mutation Nature mutation  Valeur fonciere No voie B/T/Q Type de voie Code voie              Voie Code postal    Commune Code departement Code commune Prefixe de section Section No plan No Volume 1er lot Surface Carrez du 1er lot 2eme lot Surface Carrez du 2eme lot 3eme lot Surface Carrez du 3eme lot 4eme lot Surface Carrez du 4eme lot 5eme lot Surface Carrez du 5eme lot Nombre de lots Code type local  Type local Identifiant local  Surface reelle bati  Nombre pieces principales Nature culture Nature culture speciale  Surface terrain
0                     NaN                NaN            NaN            NaN            NaN            NaN            NaN         000001    2025-01-07           Vente         468000.0     NaN   NaN          NaN      B078            FARGES        1550     FARGES               01          158                NaN       B     8

In [4]:
# --- CREATION D'UN DATAFRAME REDUIT ---

cols = [
    "Date mutation",
    "Valeur fonciere",
    "Surface reelle bati",
    "Surface terrain",
    "Nombre pieces principales",
    "Type local",
    "Voie",
    "Code postal",
    "Commune",
    "Code departement",
]

df2 = df[cols].copy() #on crée un autre dataframe avec un nombre réduit de colonnes
print(df2.head()) 

  Date mutation  Valeur fonciere  Surface reelle bati  Surface terrain  Nombre pieces principales  Type local              Voie Code postal    Commune Code departement
0    2025-01-07         468000.0                  NaN             78.0                        NaN         NaN            FARGES        1550     FARGES               01
1    2025-01-07         468000.0                111.0            133.0                        5.0      Maison  DE LA REPUBLIQUE        1550     FARGES               01
2    2025-01-07         468000.0                  0.0            133.0                        0.0  Dépendance  DE LA REPUBLIQUE        1550     FARGES               01
3    2025-01-06         180000.0                  NaN             46.0                        NaN         NaN        LE VILLAGE        1200  MONTANGES               01
4    2025-01-06         180000.0                  NaN             17.0                        NaN         NaN        LE VILLAGE        1200  MONTANGES          

In [5]:
df2.describe()

,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales
count,1387077,1.373082e+06,813890.000000,9.394530e+05,813890.000000
mean,2025-03-23 16:13:00.920453376,7.034421e+05,64.034367,3.034890e+03,1.707985
min,2025-01-01 00:00:00,1.500000e-01,0.000000,0.000000e+00,0.000000
25%,2025-02-13 00:00:00,5.935000e+04,0.000000,2.520000e+02,0.000000
50%,2025-03-24 00:00:00,1.580000e+05,28.000000,6.390000e+02,0.000000
75%,2025-04-28 00:00:00,2.950000e+05,80.000000,1.905000e+03,3.000000
max,2025-06-30 00:00:00,3.172908e+08,303623.000000,4.353772e+07,83.000000
std,NaN,4.944190e+06,635.390947,4.940377e+04,2.061545


In [6]:
# --- NETTOYAGE BASE DE DONNEES ---

# Variables numériques à nettoyer (valeurs extrêmes)
cols = ["Valeur fonciere", "Surface reelle bati", "Surface terrain", "Nombre pieces principales"]

# Calcul des seuils 5% et 95% pour chaque variable
low_quantiles = df2[cols].quantile(0.05)
high_quantiles = df2[cols].quantile(0.95)

# Filtrage : on garde uniquement les valeurs comprises entre les percentiles 5% et 95%
for col in cols:
    df2 = df2[(df2[col] >= low_quantiles[col]) & (df2[col] <= high_quantiles[col])]

df2.describe()

,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales
count,331047,3.310470e+05,331047.000000,331047.000000,331047.000000
mean,2025-03-21 22:40:34.059212288,2.433279e+05,49.344302,746.493990,2.018191
min,2025-01-01 00:00:00,2.300000e+03,0.000000,33.000000,0.000000
25%,2025-02-13 00:00:00,1.200000e+05,0.000000,251.000000,0.000000
50%,2025-03-20 00:00:00,2.000000e+05,50.000000,500.000000,2.000000
75%,2025-04-25 00:00:00,3.150000e+05,90.000000,845.000000,4.000000
max,2025-06-30 00:00:00,1.045000e+06,151.000000,12312.000000,5.000000
std,NaN,1.801918e+05,46.906374,1024.329696,1.957335


## Partie 2 : création des features essentielles

In [7]:
df2["Valeur_m2"] = df2["Valeur fonciere"] / df2["Surface reelle bati"]
df2["Annee"] = df2["Date mutation"].dt.year
df2["Mois"] = df2["Date mutation"].dt.month

In [11]:
# XXX CREATION DE L'ADRESSE XXX

df2[["Voie", "Code postal", "Commune"]].isna().sum()


Voie            2
Code postal    23
Commune         0
dtype: int64

In [12]:
df2["adresse"] = (
    df2["Voie"].fillna("") +
    ", " +
    df2["Code postal"].fillna("").astype(str) +
    " " +
    df2["Commune"].fillna("")
)

df2["adresse"].head(20)


1               DE LA REPUBLIQUE, 1550 FARGES
5             DE LA FRUITIERE, 1200 MONTANGES
9             DU PETIT CORGENON, 1310 BUELLAS
13            DU PETIT CORGENON, 1310 BUELLAS
21                  DE LA FRAZE, 1990 BANEINS
30             DE L' ETANG RATIER, 1390 RANCE
32             DE L' ETANG RATIER, 1390 RANCE
35                  DES MORILLES, 1710 THOIRY
38                    VIE CREUSE, 1550 POUGNY
45    JEAN LOUIS MASSOT, 1000 BOURG-EN-BRESSE
52                   DES PIVOINES, 1210 ORNEX
54                   DES PIVOINES, 1210 ORNEX
65               DE GROISSIAT, 1100 BELLIGNAT
73              DES CONDAMINES, 1340 FOISSIAT
74              DES CONDAMINES, 1340 FOISSIAT
75                DE CHAMANDRE, 1340 FOISSIAT
79                   LES MONTETS, 1560 CORMOZ
81           DU CHATEAU D EAU, 1800 MEXIMIEUX
84             DES CHARMETTES, 1310 CURTAFOND
88                      LEYMENT, 1150 LAGNIEU
Name: adresse, dtype: object

In [13]:
# XXX VERIFICATIONS DE l'ABSENCE D'INCOHERENCE XXX

df2[df2["adresse"].str.startswith(",")].head() # vérifier s’il existe des adresses du type : ", 1550 FARGES"
df2[df2["adresse"].str.contains("  ")].head() # détecter les adresses contenant deux espaces consécutifs : " "



,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales,Type local,Voie,Code postal,Commune,Code departement,Valeur_m2,Annee,Mois,adresse
181893,2025-02-25,375000.0,129.0,657.0,4.0,Maison,DU GENERAL DE GAULLE,14990,BERNIERES-SUR-MER,14,2906.976744,2025,2,"DU GENERAL DE GAULLE, 14990 BERNIERES-SUR-MER"
538444,2025-01-13,467400.0,112.0,659.0,4.0,Maison,DE MONTEUIL,NaN,LA BUISSE,38,4173.214286,2025,1,"DE MONTEUIL, LA BUISSE"
683080,2025-05-05,60000.0,98.0,1793.0,4.0,Maison,DU CHASTELAS,NaN,MONT LOZERE ET GOULET,48,612.244898,2025,5,"DU CHASTELAS, MONT LOZERE ET GOULET"
683865,2025-06-26,316250.0,130.0,2725.0,5.0,Maison,DES BERGES,NaN,GORGES DU TARN CAUSSES,48,2432.692308,2025,6,"DES BERGES, GORGES DU TARN CAUSSES"
683870,2025-06-26,316250.0,130.0,2725.0,5.0,Maison,DES BERGES,NaN,GORGES DU TARN CAUSSES,48,2432.692308,2025,6,"DES BERGES, GORGES DU TARN CAUSSES"


In [14]:
# on remplace ces "  " par " "
df2["adresse"] = df2["adresse"].str.replace("  ", " ", regex=False)

df2[df2["adresse"].str.contains("  ")].head() # détecter les adresses contenant deux espaces consécutifs : " "


,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales,Type local,Voie,Code postal,Commune,Code departement,Valeur_m2,Annee,Mois,adresse
993857,2025-02-21,388000.0,160.0,228.0,5.0,Maison,DU CHEDOUET ROULLEE,72600,VILLENEUVE EN PERSEIGNE,72,2425.00,2025,2,"DU CHEDOUET ROULLEE, 72600 VILLENEUVE EN PERS..."
993859,2025-02-21,388000.0,50.0,228.0,1.0,Maison,DU CHEDOUET ROULLEE,72600,VILLENEUVE EN PERSEIGNE,72,7760.00,2025,2,"DU CHEDOUET ROULLEE, 72600 VILLENEUVE EN PERS..."
993861,2025-02-21,388000.0,50.0,16037.0,1.0,Maison,DU CHEDOUET ROULLEE,72600,VILLENEUVE EN PERSEIGNE,72,7760.00,2025,2,"DU CHEDOUET ROULLEE, 72600 VILLENEUVE EN PERS..."
993862,2025-02-21,388000.0,160.0,16037.0,5.0,Maison,DU CHEDOUET ROULLEE,72600,VILLENEUVE EN PERSEIGNE,72,2425.00,2025,2,"DU CHEDOUET ROULLEE, 72600 VILLENEUVE EN PERS..."
1019052,2025-06-26,775000.0,160.0,674.0,5.0,Maison,SAINT BERNARD MACOT,73210,LA PLAGNE TARENTAISE,73,4843.75,2025,6,"SAINT BERNARD MACOT, 73210 LA PLAGNE TARENT..."


In [15]:
# XXX GEOCODAGE XXX

%pip install geopy tqdm


Note: you may need to restart the kernel to use updated packages.


In [16]:
# import des librairies
from geopy.geocoders import BANFrance #BAN France convertit les adresses en coordonnées
import pandas as pd # pour manipuler les tableaux




In [17]:
geolocator = BANFrance() #création d'un objet geolocator qui sait parler à BAN France. geolocator.geocode("DE LA REPUBLIQUE, 1550 FARGES")

def geocode_address(adresse): #fonction qui renvoie les coordonnées ou None si erreur
    try:
        result = geolocator.geocode(adresse)
        if result:
            return result.latitude, result.longitude
        else:
            return None, None
    except:
        return None, None


## Version de géocodage optimisée

In [18]:
sample = df2.sample(10, random_state=42)
sample


,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales,Type local,Voie,Code postal,Commune,Code departement,Valeur_m2,Annee,Mois,adresse
39934,2025-02-10,190000.0,72.0,308.0,3.0,Maison,DE SAINT JURS,4270,BRAS-D ASSE,04,2638.888889,2025,2,"DE SAINT JURS, 4270 BRAS-D ASSE"
831982,2025-06-25,157500.0,96.0,334.0,5.0,Maison,DU DOCTEUR NICK,59540,BEAUMONT-EN-CAMBRESIS,59,1640.625000,2025,6,"DU DOCTEUR NICK, 59540 BEAUMONT-EN-CAMBRESIS"
1296608,2025-01-17,822000.0,91.0,3120.0,4.0,Maison,FRANCOIS ARAGO,93100,MONTREUIL,93,9032.967033,2025,1,"FRANCOIS ARAGO, 93100 MONTREUIL"
1257204,2025-01-24,385000.0,162.0,500.0,6.0,Maison,DES EGRINS,90350,EVETTE-SALBERT,90,2376.543210,2025,1,"DES EGRINS, 90350 EVETTE-SALBERT"
147377,2025-03-18,600000.0,50.0,148.0,2.0,Appartement,BERNARD DU BOIS,13001,MARSEILLE 1ER,13,12000.000000,2025,3,"BERNARD DU BOIS, 13001 MARSEILLE 1ER"
53001,2025-02-20,600000.0,194.0,1065.0,7.0,Maison,DE TRALATORRE,6690,TOURRETTE-LEVENS,06,3092.783505,2025,2,"DE TRALATORRE, 6690 TOURRETTE-LEVENS"
851829,2025-03-26,111000.0,46.0,739.0,3.0,Appartement,DE SAINTE ANNE,61400,COURGEON,61,2413.043478,2025,3,"DE SAINTE ANNE, 61400 COURGEON"
747643,2025-02-26,342000.0,117.0,708.0,7.0,Maison,BOFFRAND,54300,LUNEVILLE,54,2923.076923,2025,2,"BOFFRAND, 54300 LUNEVILLE"
1129947,2025-06-13,199000.0,127.0,1506.0,4.0,Maison,DE LA VALLEE,80132,HUCHENNEVILLE,80,1566.929134,2025,6,"DE LA VALLEE, 80132 HUCHENNEVILLE"
1093698,2025-02-12,700000.0,150.0,445.0,5.0,Maison,DES VERGERS,78120,RAMBOUILLET,78,4666.666667,2025,2,"DES VERGERS, 78120 RAMBOUILLET"


In [19]:
# --- IMPORT DES LIBRAIRIES --

import json # lire et écrire fichier cache
import os # lire et écrire fichier cache 
from geopy.geocoders import BANFrance #géocodeur officiel
from tqdm import tqdm # barre de progression


In [20]:
geolocator = BANFrance() # création du géocodeur



In [21]:
CACHE_FILE = "geocode_cache.json" 

if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        cache = json.load(f)
else:
    cache = {}


In [22]:
def geocode_address(adresse):
    # 1. Si l’adresse est déjà connue -> on la renvoie directement
    if adresse in cache:
        return cache[adresse]

    # 2. Sinon on appelle BAN France
    try:
        result = geolocator.geocode(adresse)
        if result:
            coords = (result.latitude, result.longitude)
        else:
            coords = (None, None)
    except:
        coords = (None, None)

    # 3. On enregistre dans le cache
    cache[adresse] = coords
    return coords


In [23]:
unique_addresses = sample["adresse"].unique()


In [24]:
for adresse in tqdm(unique_addresses):
    geocode_address(adresse)


100%|██████████| 10/10 [00:00<00:00, 10.65it/s]


In [25]:
sample["lat"], sample["lon"] = zip(*sample["adresse"].apply(geocode_address))
sample


,Date mutation,Valeur fonciere,Surface reelle bati,Surface terrain,Nombre pieces principales,Type local,Voie,Code postal,Commune,Code departement,Valeur_m2,Annee,Mois,adresse,lat,lon
39934,2025-02-10,190000.0,72.0,308.0,3.0,Maison,DE SAINT JURS,4270,BRAS-D ASSE,04,2638.888889,2025,2,"DE SAINT JURS, 4270 BRAS-D ASSE",43.921154,6.136701
831982,2025-06-25,157500.0,96.0,334.0,5.0,Maison,DU DOCTEUR NICK,59540,BEAUMONT-EN-CAMBRESIS,59,1640.625000,2025,6,"DU DOCTEUR NICK, 59540 BEAUMONT-EN-CAMBRESIS",50.123214,3.451697
1296608,2025-01-17,822000.0,91.0,3120.0,4.0,Maison,FRANCOIS ARAGO,93100,MONTREUIL,93,9032.967033,2025,1,"FRANCOIS ARAGO, 93100 MONTREUIL",48.853630,2.427752
1257204,2025-01-24,385000.0,162.0,500.0,6.0,Maison,DES EGRINS,90350,EVETTE-SALBERT,90,2376.543210,2025,1,"DES EGRINS, 90350 EVETTE-SALBERT",47.671215,6.786577
147377,2025-03-18,600000.0,50.0,148.0,2.0,Appartement,BERNARD DU BOIS,13001,MARSEILLE 1ER,13,12000.000000,2025,3,"BERNARD DU BOIS, 13001 MARSEILLE 1ER",43.301811,5.377745
53001,2025-02-20,600000.0,194.0,1065.0,7.0,Maison,DE TRALATORRE,6690,TOURRETTE-LEVENS,06,3092.783505,2025,2,"DE TRALATORRE, 6690 TOURRETTE-LEVENS",43.778902,7.283266
851829,2025-03-26,111000.0,46.0,739.0,3.0,Appartement,DE SAINTE ANNE,61400,COURGEON,61,2413.043478,2025,3,"DE SAINTE ANNE, 61400 COURGEON",48.478219,0.612615
747643,2025-02-26,342000.0,117.0,708.0,7.0,Maison,BOFFRAND,54300,LUNEVILLE,54,2923.076923,2025,2,"BOFFRAND, 54300 LUNEVILLE",48.585173,6.497680
1129947,2025-06-13,199000.0,127.0,1506.0,4.0,Maison,DE LA VALLEE,80132,HUCHENNEVILLE,80,1566.929134,2025,6,"DE LA VALLEE, 80132 HUCHENNEVILLE",50.040507,1.792018
1093698,2025-02-12,700000.0,150.0,445.0,5.0,Maison,DES VERGERS,78120,RAMBOUILLET,78,4666.666667,2025,2,"DES VERGERS, 78120 RAMBOUILLET",48.638194,1.835279


In [26]:
# XXX NORMALISATION DU DATASET XXX

df_maison = df2[df2["Type local"] == "Maison"]
df_appart = df2[df2["Type local"] == "Appartement"]
